## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from keras import regularizers


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [4]:
classifier=Sequential()

#卷積組合
classifier.add(Conv2D(32, kernel_size=(3, 3), padding='same', input_shape=(32,32,3), activation='relu'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

#自己決定MaxPooling2D放在哪裡
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(100,activation='relu'))##regularizers
classifier.add(BatchNormalization()) ##BatchNormalization
#output_dim=100,activation=relu

#輸出
classifier.add(Dense(10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

Epoch 1/100
50000/50000 [==============================] - 10s - loss: 1.2360 - acc: 0.5643    
Epoch 2/100
50000/50000 [==============================] - 6s - loss: 0.8841 - acc: 0.6899     
Epoch 3/100
50000/50000 [==============================] - 6s - loss: 0.7404 - acc: 0.7432     
Epoch 4/100
50000/50000 [==============================] - 6s - loss: 0.6331 - acc: 0.7802     
Epoch 5/100
50000/50000 [==============================] - 6s - loss: 0.5395 - acc: 0.8142     
Epoch 6/100
50000/50000 [==============================] - 6s - loss: 0.4591 - acc: 0.8417     
Epoch 7/100
50000/50000 [==============================] - 6s - loss: 0.3847 - acc: 0.8684     
Epoch 8/100
50000/50000 [==============================] - 6s - loss: 0.3118 - acc: 0.8946     
Epoch 9/100
50000/50000 [==============================] - 6s - loss: 0.2644 - acc: 0.9104     
Epoch 10/100
50000/50000 [==============================] - 6s - loss: 0.2089 - acc: 0.9312     
Epoch 11/100
50000/50000 [=============

50000/50000 [==============================] - 6s - loss: 0.0181 - acc: 0.9937     
Epoch 86/100
50000/50000 [==============================] - 6s - loss: 0.0207 - acc: 0.9928     
Epoch 87/100
50000/50000 [==============================] - 6s - loss: 0.0183 - acc: 0.9937     
Epoch 88/100
50000/50000 [==============================] - 6s - loss: 0.0172 - acc: 0.9940     
Epoch 89/100
50000/50000 [==============================] - 6s - loss: 0.0164 - acc: 0.9941     
Epoch 90/100
50000/50000 [==============================] - 6s - loss: 0.0163 - acc: 0.9942     
Epoch 91/100
50000/50000 [==============================] - 6s - loss: 0.0154 - acc: 0.9952     
Epoch 92/100
50000/50000 [==============================] - 6s - loss: 0.0184 - acc: 0.9937     
Epoch 93/100
50000/50000 [==============================] - 6s - loss: 0.0229 - acc: 0.9924     
Epoch 94/100
50000/50000 [==============================] - 6s - loss: 0.0181 - acc: 0.9939     
Epoch 95/100
50000/50000 [=================

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [5]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[  7.19557818e-07,   8.63061318e-14,   5.73313519e-05,
          9.77964401e-01,   2.19766013e-02,   2.29512537e-11,
          1.73284642e-09,   1.46186549e-10,   9.41228393e-07,
          7.19521867e-19]], dtype=float32)